In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
import xml.etree.ElementTree as ET
import pandas as pd
from operator import itemgetter
from sklearn.preprocessing import OneHotEncoder

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def form_getter(file_list, path):
    master_list = []

    for i in tqdm(file_list):
        root = ET.parse(i).getroot()
        track = root[0]
        date = root[1]
        for race in root.iter("Race"):
            print(race)
            ids = []
            boxes = []
            weights = []
            dog_name = []
            for dog in race.iter("Dog"):
                if dog[0].text not in "12345678":
                    print("longscratch")
                    continue
                else:
                    print(dog[0].text)


                if not dog.attrib["id"]:
                    continue
                if not dog[0].text:
                    continue
                if (dog[0].text in "RSBFNPTBD") or ("=" in dog[0].text):
                    # print("scratched check working")
                    continue
                else:
                    ids.append(dog.attrib["id"])
                    boxes.append(dog[0].text)
                    weights.append(dog[5].text)
                    dog_name.append(dog[1].text)


            zipall = list(
                zip(
                    ids,
                    boxes,
                    weights,
                    dog_name
                )
            )
            # print(zipall)
            for i in range(len(zipall)):
                zipall[i] = (
                    race.attrib["id"],
                    date.text,
                    track.text,
                    race[3].text,
                    race[4].text,
                ) + zipall[i]
                master_list.append(zipall[i])

    return master_list


In [3]:
def track_id_gen(dist, track):
    return track[0:4]+"-"+str(int(dist))

In [4]:
#pickle Load results
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
form_file = open( 'results-df-merged-prices.npy', 'rb')
form = pickle.load(form_file)
form

,race_id,date,track,dist_x,grade,place,dog_id,box,split_times,split_margins,...,PPMAX,PPMIN,IPMAX,IPMIN,MORNINGTRADEDVOL,PPTRADEDVOL,IPTRADEDVOL,<!DOCTYPE html>,dog,loc
0,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,1.0,1296100098,2,6.73,0.00,...,1.0,1001.0,1.0,1001.0,171.76,0.0,0.0,NaN,Swift Operator,AUS
1,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,2.0,1177320005,3,6.87,0.14,...,1.0,1001.0,1.0,1001.0,391.54,0.0,0.0,NaN,Last Princess,AUS
2,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,3.0,1549980026,5,6.99,0.26,...,1.0,1001.0,1.0,1001.0,663.20,0.0,0.0,NaN,Daintree Prinz,AUS
3,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,4.0,1474670055,7,6.88,0.15,...,1.0,1001.0,1.0,1001.0,1406.34,0.0,0.0,NaN,Tic Toc Toby,AUS
4,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,5.0,913850094,8,6.91,0.18,...,1.0,1001.0,1.0,1001.0,119.00,0.0,0.0,NaN,Race Roquette,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530019,224547720,30 May 17,Warragul,680.0,Special Event,3.0,954250096,2,18.15,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530020,224547720,30 May 17,Warragul,680.0,Special Event,4.0,1428630000,3,19.10,0.95,...,1.0,1001.0,1.0,1001.0,7248.84,0.0,0.0,NaN,Ebby Ripper,AUS
530021,224547720,30 May 17,Warragul,680.0,Special Event,5.0,-743571,4,18.83,0.68,...,1.0,1001.0,1.0,1001.0,151.40,0.0,0.0,NaN,Dewana Babe,AUS
530022,224547720,30 May 17,Warragul,680.0,Special Event,6.0,-748349,6,19.03,0.88,...,1.0,1001.0,1.0,1001.0,96.44,0.0,0.0,NaN,Always About Me,AUS


In [5]:
tracks = form.track.unique().tolist()

ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit_transform(form[['track']])
#enc = OneHotEncoder( categories=tracks)


In [6]:
file_list = ["ff-SAN01Sep22.xml"]
fg = form_getter(file_list, path="")
fg

100%|██████████| 1/1 [00:00<00:00, 39.99it/s]

<Element 'Race' at 0x000001D3D4214810>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3EE4BDE50>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3EE5294F0>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3EE5783B0>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3EE5C8090>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3EE635EF0>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3F07E5E00>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3F0847400>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3F08A5950>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3F08FBC70>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3F096C900>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x000001D3F09DF450>
1
2
3
4
5
6
7
8
longscratch
longscratch


[('820735752',
  '01 Sep 22',
  'Sandown Park',
  '01 Sep 22 08:10AM',
  '515m',
  '640983411',
  '1',
  'Trk/Dst 1-0-0-1',
  'CRUISE MODE'),
 ('820735752',
  '01 Sep 22',
  'Sandown Park',
  '01 Sep 22 08:10AM',
  '515m',
  '575326859',
  '2',
  'Trk/Dst 1-0-1-0',
  'MANUKA BOY'),
 ('820735752',
  '01 Sep 22',
  'Sandown Park',
  '01 Sep 22 08:10AM',
  '515m',
  '570897028',
  '3',
  'Trk/Dst 3-0-1-0',
  'PARUMBA GEM'),
 ('820735752',
  '01 Sep 22',
  'Sandown Park',
  '01 Sep 22 08:10AM',
  '515m',
  '649812534',
  '4',
  'Trk/Dst 1-0-0-1',
  'HURRICANE DEMON'),
 ('820735752',
  '01 Sep 22',
  'Sandown Park',
  '01 Sep 22 08:10AM',
  '515m',
  '662302469',
  '5',
  'Trk/Dst 1-1-0-0',
  'CHEEKY WINK'),
 ('820735752',
  '01 Sep 22',
  'Sandown Park',
  '01 Sep 22 08:10AM',
  '515m',
  '623911850',
  '6',
  'Trk/Dst 1-1-0-0',
  'MEPUNGA TILLY'),
 ('820735752',
  '01 Sep 22',
  'Sandown Park',
  '01 Sep 22 08:10AM',
  '515m',
  '662599554',
  '7',
  'Trk/Dst 1-0-1-0',
  'MINTER FROST'),


In [9]:
pred_df = pd.DataFrame(fg, columns=["race_id","dateF", "track", "datetime", "dist", "dog_id", "box", "extra1", "dogname"])
pred_df["dist"] = pred_df["dist"].astype(str).str[:-1].astype(float)
pred_df

,race_id,dateF,track,datetime,dist,dog_id,box,extra1,dogname
0,820735752,01 Sep 22,Sandown Park,01 Sep 22 08:10AM,515.0,640983411,1,Trk/Dst 1-0-0-1,CRUISE MODE
1,820735752,01 Sep 22,Sandown Park,01 Sep 22 08:10AM,515.0,575326859,2,Trk/Dst 1-0-1-0,MANUKA BOY
2,820735752,01 Sep 22,Sandown Park,01 Sep 22 08:10AM,515.0,570897028,3,Trk/Dst 3-0-1-0,PARUMBA GEM
3,820735752,01 Sep 22,Sandown Park,01 Sep 22 08:10AM,515.0,649812534,4,Trk/Dst 1-0-0-1,HURRICANE DEMON
4,820735752,01 Sep 22,Sandown Park,01 Sep 22 08:10AM,515.0,662302469,5,Trk/Dst 1-1-0-0,CHEEKY WINK
...,...,...,...,...,...,...,...,...,...
86,821751728,01 Sep 22,Sandown Park,01 Sep 22 12:13PM,515.0,312553983,4,Trk/Dst 59-3-6-16,ARLINGTON
87,821751728,01 Sep 22,Sandown Park,01 Sep 22 12:13PM,515.0,565382028,5,Trk/Dst 0-0-0-0,REINFORCEMENTS
88,821751728,01 Sep 22,Sandown Park,01 Sep 22 12:13PM,515.0,461114954,6,Trk/Dst 3-0-0-0,DOONSIDE SHADOW
89,821751728,01 Sep 22,Sandown Park,01 Sep 22 12:13PM,515.0,552236784,7,Trk/Dst 1-1-0-0,EXTREME FORCE


In [10]:
pred_df.columns

ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit(form[['track']])

transformed.transform(pred_df[['track']])

pred_df["tracksOnehot"] =  transformed.transform(pred_df[['track']]).tolist()
pred_df["tracksOnehot"]

0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
3     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
4     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
                          ...                      
86    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
87    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
88    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
89    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
90    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Name: tracksOnehot, Length: 91, dtype: object

In [11]:
race_forms = form.groupby(["race_id"])
dog_forms = form.groupby(["dog_id"])
print(race_forms)
race_input_layer = []
race_classes = []
full_details = []

race_pred_forms = pred_df.groupby(["race_id"])

for i, j in tqdm(race_forms):
    print( f"{i=},{j=}")
    break


for i, j in tqdm(race_pred_forms):
    try:
        race_date = pd.to_datetime(j["dateF"].iloc[0])
        # print(f"{race_date=}")
        dogs = j.dog_id
        # prev_form = form[(form["date"]< race_date)]
        race_id = i
        # print(prev_form.shape)
        race_dist = j.dist.iloc[1]
        trackOHE = j.tracksOnehot.iloc[1]
        dog_info = []


        i = 0
        for dog in dogs:
            # print(f"dog id = {dog}")
            prev_form = dog_forms.get_group(dog)
            prev_form = prev_form[(prev_form["dateF"] < race_date)]
            dog_speed_avg = prev_form.speed.mean()
            dog_speed_sd = prev_form.speed.std()
            dog_speed_max = prev_form.speed.max()
            dog_split_avg = prev_form.split_speed.mean()
            dog_split_sd = prev_form.split_speed.std()
            dog_split_max = prev_form.split_speed.max()
            dog_split_mg_avg = prev_form.split_margins.mean()
            dog_margin_mean = prev_form.margin.mean()
            dog_margin_sd = prev_form.margin.std()
            dog_race_count = prev_form.shape[0]
            dog_wins = prev_form[prev_form["place"] == 1].shape[0]
            dog_places = prev_form[prev_form["place"] <= 3].shape[0]
            dog_box = j.box.iloc[i]
            dog_weight = prev_form.weigth.iloc[0]
            dog_stats = (
                dog,
                [
                    dog_box,
                    dog_speed_avg,
                    dog_speed_sd,
                    dog_speed_max,
                    dog_split_avg,
                    dog_split_sd,
                    dog_split_max,
                    dog_margin_mean,
                    dog_margin_sd,
                    dog_race_count,
                    dog_wins,
                    dog_places,
                    dog_weight,
                ],
            )
            dog_info.append(dog_stats)
            # print(f"{dog_stats=}")
            i = i + 1

        dog_info.sort(key=itemgetter(0))
        input_layer = (
            trackOHE
            + [race_dist]
            + [item for sublist in [x[1] for x in dog_info] for item in sublist]
        )
        if len(dogs) > 8:
            print(f"{race_id=}")
            print(f"{race_date=}")
            print(f"{j.track.iloc[1]=}")
        race_input_layer.append(input_layer)
        full_details.append(
            (race_id, input_layer))
    except Exception as e:
        print(f"{e=}")


  0%|          | 0/12 [00:00<?, ?it/s]

i='12539914',j=         race_id       date      track  dist_x         grade  place  \
444750  12539914  22 Jun 18  Traralgon   525.0  Maiden Final    1.0   
444751  12539914  22 Jun 18  Traralgon   525.0  Maiden Final    2.0   
444752  12539914  22 Jun 18  Traralgon   525.0  Maiden Final    3.0   
444753  12539914  22 Jun 18  Traralgon   525.0  Maiden Final    4.0   
444754  12539914  22 Jun 18  Traralgon   525.0  Maiden Final    5.0   
444755  12539914  22 Jun 18  Traralgon   525.0  Maiden Final    6.0   
444756  12539914  22 Jun 18  Traralgon   525.0  Maiden Final    7.0   
444757  12539914  22 Jun 18  Traralgon   525.0  Maiden Final    8.0   

            dog_id  box split_times  split_margins  ...  PPMAX   PPMIN  IPMAX  \
444750   115367782    8        4.86           0.00  ...    1.0  1001.0    1.0   
444751   128006011    5        5.04           0.18  ...    1.0  1001.0    1.0   
444752   167542442    4        4.90           0.04  ...    NaN     NaN    NaN   
444753   145856635   

100%|██████████| 12/12 [00:00<00:00, 18.90it/s]


In [72]:
with open("prediction_inputs.npy", "wb") as fp:   #Pickling
    
    pickle.dump(full_details, fp)

In [12]:
full_details

[('820735752',
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   515.0,
   '3',
   0.05895948472696574,
   0.0008553943770938378,
   0.0606990291262136,
   0.05939257461692149,
   0.0006027420277007245,
   0.06071290065087939,
   8.7475,
   4.922421490824748,
   16,
   0,
   3,
   '25.8',
   '2',
   0.05882297734627832,
   0.000510140035586804,
   0.059375,
   0.05942143446827861,
   0.00026240645703854044,
   0.05972047327487575,
   7.466666666666666,
   7.738554990005132,
   3,
   0,
   2,
   '34.9',
   '6',
   0.05881589148956331,
   0.0011413817545716676,
   0.06015533980582524,
   0.05832591826716292,
   0.0007282929407247996,
   0.059085517607338524,
   7.4485714285714275,
   7.973507025501031,
   7,
   1,
   3,
   '29.3',
   '1',
   0.05812294850147741,
   0.0008570038900407612,
   0.059225,
   0.058686244405054586,
   0.0015211815317792126,
   0.06107467901807021,
   2.752,
   1.4575904774661503,
   5,
   0,
   3,
   '27.8',
   '4',
  